In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '6'
import numpy as np
from PIL import Image
import time
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import subprocess
import tifffile
import torch
import segmentation_models_pytorch as smp
import torch.nn.functional as F
import cv2
import re

/gris/gris-f/homelv/tdou/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class SemicolDataset(Dataset):
    def __init__(self, root_dirs, patch_size, overlap, display=False, output_dir=None):
        self.root_dirs = root_dirs
        self.patch_size = patch_size
        self.overlap = overlap
        self.display = display
        self.output_dir = output_dir
        self.file_list = self.get_file_list()


    def get_file_list(self):
        file_list = []
        for root_dir in self.root_dirs:
            for case_folder in os.listdir(root_dir):
                case_folder_path = os.path.join(root_dir, case_folder)                
                if os.path.isdir(case_folder_path):
                    image_dir = os.path.join(case_folder_path, 'image')
                    mask_dir = os.path.join(case_folder_path, 'mask')
                    
                    image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.tif')]
                    mask_files = [os.path.join(mask_dir, f) for f in os.listdir(mask_dir) if f.endswith('.tif')]

                    # Create a dictionary with the shared part of the filename as the key
                    image_dict = {re.search(r'(.*).tif', os.path.basename(img_file)).group(1): img_file for img_file in image_files}
                    mask_dict = {re.search(r'(.*).tif', os.path.basename(mask_file)).group(1).replace('-labelled', ''): mask_file for mask_file in mask_files}

                    # Now sort the keys and create the file list
                    for key in sorted(image_dict.keys()):
                        if key in mask_dict: # ensure that each image has a corresponding mask
                            file_list.append((image_dict[key], mask_dict[key], case_folder))

        return file_list


    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_name, mask_name, case_folder = self.file_list[idx]
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_name)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        
        mask_patches, patch_indices = self.get_patches(mask, mask_name, case_folder, 'mask', return_indices=True)
        image_patches = self.get_patches(image, img_name, case_folder, 'image', indices=patch_indices)
    
        return image_patches, mask_patches

    def get_patches(self, img, img_name, case_folder, patch_type, return_indices=False, indices=None):
        patches=[]
        indices_list = []
        for i in range(0, img.shape[0], self.patch_size - self.overlap):  
            for j in range(0, img.shape[1], self.patch_size - self.overlap):
                patch_h = min(self.patch_size, img.shape[0] - i)  
                patch_w = min(self.patch_size, img.shape[1] - j)
                if i+patch_h >= img.shape[0] and j+patch_w >= img.shape[1]:#control border.
                    patch = img[i+patch_h-self.patch_size:i+patch_h, j+patch_w-self.patch_size: j + patch_w]
                elif i+patch_h >= img.shape[0] and j+patch_w < img.shape[1]:
                    patch = img[i + patch_h - self.patch_size:i + patch_h, j:j + patch_w]
                elif i+patch_h < img.shape[0] and j+patch_w >= img.shape[1]:
                    patch = img[i :i + patch_h, j+patch_w-self.patch_size: j + patch_w]
                else:
                    patch = img[i:i + patch_h, j:j + patch_w]
                
                if patch_type == 'mask':
                    if not np.all(patch == [255, 255, 255]): 
                        self.save_patch(patch, i, j, img_name, patch_type)
                        patches.append(patch)
                        indices_list.append((i, j))
                elif patch_type == 'image':
                    if (i, j) in indices:
                        self.save_patch(patch, i, j, img_name, patch_type)
                        patches.append(patch)
                
        if return_indices:
            return patches, indices_list
        else:
            return patches

    
    def save_patch(self, patch, i, j, img_name, patch_type):
        if self.display:
            img_basename = os.path.basename(img_name)
            # use name without expand name
            img_name_without_ext = os.path.splitext(img_basename)[0]

            # assign to the respective folders
            if patch_type == 'image':
                patch_dir = os.path.join(self.output_dir, 'image')
            elif patch_type == 'mask':
                patch_dir = os.path.join(self.output_dir, 'mask')

            os.makedirs(patch_dir, exist_ok=True)
            save_path = os.path.join(patch_dir, f"{img_name_without_ext}_{i}_{j}.tif")
            patch_image = Image.fromarray(patch)
            patch_image.save(save_path)

In [3]:
root_dirs = [
    '/local/scratch/rwang/patch/DS_M_1',
    '/local/scratch/rwang/patch/DS_M_2'
]

In [4]:
dataset = SemicolDataset(root_dirs=root_dirs, patch_size=256, overlap=100, display=True, output_dir="/local/scratch/TuoDou/")

dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=10)
for i, patches in enumerate(dataloader):
    print(f"Processed image {i+1}")
    

Processed image 1
Processed image 2
Processed image 3
Processed image 4
Processed image 5
Processed image 6
Processed image 7
Processed image 8
Processed image 9
Processed image 10
Processed image 11
Processed image 12
Processed image 13
Processed image 14
Processed image 15
Processed image 16
Processed image 17
Processed image 18
Processed image 19
Processed image 20
Processed image 21
Processed image 22
Processed image 23
Processed image 24
Processed image 25
Processed image 26
Processed image 27
Processed image 28
Processed image 29
Processed image 30
Processed image 31
Processed image 32
Processed image 33
Processed image 34
Processed image 35
Processed image 36
Processed image 37
Processed image 38
Processed image 39
Processed image 40
Processed image 41
Processed image 42
Processed image 43
Processed image 44
Processed image 45
Processed image 46
Processed image 47
Processed image 48
Processed image 49
Processed image 50
Processed image 51
Processed image 52
Processed image 53
Pr

In [ ]:
import os
import re

# 这是你的目标目录
directory = r"/local/scratch/TuoDou/mask"

# 在目录中迭代每个文件
for filename in os.listdir(directory):
    # 检查文件名是否匹配你的模式
    match = re.match(r'^(.*?)-labelled_(.*?).tif$', filename)
    if match:
        # 如果匹配，重新构造文件名
        new_filename = f"{match.group(1)}_{match.group(2)}-labelled.tif"
        # 重命名文件
        os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
